#저번에 했던게 용량을 너무 차지해서 쉬운방법찾아서 하는것

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 79970, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 79970 (delta 60), reused 140 (delta 58), pack-reused 79824
Receiving objects: 100% (79970/79970), 594.19 MiB | 24.72 MiB/s, done.
Resolving deltas: 100% (56903/56903), done.


In [ ]:
%%bash
sudo apt install -y protobuf-compiler
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cd /content
git clone https://github.com/cocodataset/cocoapi.git
cd /content/cocoapi/PythonAPI
make
cp -r pycocotools /content/models/research/
cd /content/models/research/
cp object_detection/packages/tf2/setup.py .
python -m pip install .
cd /content

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

#안돌려도 된다 Proceed (y/n)에 빈칸나오면 y적고 엔터눌러라

In [14]:
!pip uninstall imgaug
!pip install imgaug==0.4.0

Found existing installation: imgaug 0.4.0
Uninstalling imgaug-0.4.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/imgaug-0.4.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/imgaug/*
Proceed (y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n
Proceed (y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n
Proceed (y/n)? y
  Successfully uninstalled imgaug-0.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 4.1 MB/s 


In [16]:
!pip install -q tflite-model-maker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.11.0 requires tensorflow<2.12,>=2.11.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.8.4 which is incompatible.
object-detection 0.1 requires tf-models-official>=2.5.1, but you have tf-models-official 2.3.0 which is incompatible.


In [12]:
import numpy as np
import os
import shutil
import random
from lxml import etree
import re
import xml.etree.ElementTree as ET

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [15]:
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

AttributeError: ignored

In [9]:
import tensorflow as tf

In [ ]:
# from tensorflow_datasets.core.utils.lazy_imports_utils import tensorflow as tf
# import tensorflow_datasets.public_api as tfds

In [10]:
from object_detection.utils import dataset_util

In [27]:
!mkdir dataset

In [28]:
%cd dataset

/content/dataset


In [29]:
!tar -xvf /content/dataset/annotations.tar.gz

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
annotations/trimaps/._pomeranian_180.png
annotations/trimaps/pomeranian_180.png
annotations/trimaps/._pomeranian_181.png
annotations/trimaps/pomeranian_181.png
annotations/trimaps/._pomeranian_182.png
annotations/trimaps/pomeranian_182.png
annotations/trimaps/._pomeranian_183.png
annotations/trimaps/pomeranian_183.png
annotations/trimaps/._pomeranian_184.png
annotations/trimaps/pomeranian_184.png
annotations/trimaps/._pomeranian_185.png
annotations/trimaps/pomeranian_185.png
annotations/trimaps/._pomeranian_186.png
annotations/trimaps/pomeranian_186.png
annotations/trimaps/._pomeranian_187.png
annotations/trimaps/pomeranian_187.png
annotations/trimaps/._pomeranian_188.png
annotations/trimaps/pomeranian_188.png
annotations/trimaps/._pomeranian_189.png
annotations/trimaps/pomeranian_189.png
annotations/trimaps/._pomeranian_19.png
annotations/trimaps/pomeranian_19.png
annotations/trimaps/._pomeranian_190.png
annotations/trimaps/pomeranian_190.png
annota

In [30]:
!tar -xvf /content/dataset/images.tar.gz

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
images/american_bulldog_194.jpg
images/Birman_175.jpg
images/american_bulldog_178.jpg
images/staffordshire_bull_terrier_93.jpg
images/Egyptian_Mau_58.jpg
images/havanese_44.jpg
images/english_cocker_spaniel_97.jpg
images/Ragdoll_111.jpg
images/american_pit_bull_terrier_159.jpg
images/english_setter_134.jpg
images/english_setter_87.jpg
images/wheaten_terrier_198.jpg
images/pug_180.jpg
images/boxer_172.jpg
images/Sphynx_197.jpg
images/german_shorthaired_2.jpg
images/British_Shorthair_9.jpg
images/pug_89.jpg
images/japanese_chin_39.jpg
images/Ragdoll_20.jpg
images/american_bulldog_111.jpg
images/yorkshire_terrier_112.jpg
images/Persian_88.jpg
images/pug_9.jpg
images/Birman_40.jpg
images/British_Shorthair_272.jpg
images/havanese_7.jpg
images/pomeranian_154.jpg
images/beagle_49.jpg
images/British_Shorthair_94.jpg
images/Persian_90.jpg
images/staffordshire_bull_terrier_30.jpg
images/samoyed_113.jpg
images/english_cocker_spaniel_46.jpg
images/Abyssinian_22.

In [15]:
data_dir = "/content/dataset"

In [20]:
%cd /content

/content


In [31]:
image_dir = os.path.join(data_dir, 'images')
annotations_dir = os.path.join(data_dir, 'annotations')
examples_path = os.path.join(annotations_dir, 'trainval.txt')
examples_list = dataset_util.read_examples_list(examples_path)

In [32]:
print(image_dir)
print(annotations_dir)
print(examples_path)
print(examples_list)
print("examples_list size=", len(examples_list))

/content/dataset/images
/content/dataset/annotations
/content/dataset/annotations/trainval.txt
['Abyssinian_100', 'Abyssinian_101', 'Abyssinian_102', 'Abyssinian_103', 'Abyssinian_104', 'Abyssinian_105', 'Abyssinian_106', 'Abyssinian_107', 'Abyssinian_108', 'Abyssinian_109', 'Abyssinian_10', 'Abyssinian_110', 'Abyssinian_111', 'Abyssinian_112', 'Abyssinian_113', 'Abyssinian_114', 'Abyssinian_115', 'Abyssinian_116', 'Abyssinian_117', 'Abyssinian_118', 'Abyssinian_119', 'Abyssinian_11', 'Abyssinian_120', 'Abyssinian_121', 'Abyssinian_122', 'Abyssinian_123', 'Abyssinian_124', 'Abyssinian_125', 'Abyssinian_126', 'Abyssinian_127', 'Abyssinian_128', 'Abyssinian_129', 'Abyssinian_12', 'Abyssinian_130', 'Abyssinian_131', 'Abyssinian_132', 'Abyssinian_133', 'Abyssinian_134', 'Abyssinian_135', 'Abyssinian_136', 'Abyssinian_137', 'Abyssinian_138', 'Abyssinian_139', 'Abyssinian_13', 'Abyssinian_140', 'Abyssinian_141', 'Abyssinian_142', 'Abyssinian_143', 'Abyssinian_144', 'Abyssinian_145', 'america

In [33]:
# Test images are not included in the downloaded data set, so we shall perform
# our own split.
random.seed(42)
random.shuffle(examples_list)
num_examples = len(examples_list)
num_train = int(0.7 * num_examples)
train_examples = examples_list[:num_train]
val_examples = examples_list[num_train:]
print('%d training and %d validation examples.',
              len(train_examples), len(val_examples))

%d training and %d validation examples. 2576 1104


In [34]:
use_custom_dataset=True

In [35]:
dataset_is_split=True

In [36]:
if use_custom_dataset:
  
  # Your labels map as a dictionary (zero is reserved):
  label_map = {1: 'Abyssinian', 2: 'american_bulldog', 3: 'american_pit_bull_terrier', 4: 'basset_hound', 5: 'beagle',
               6: 'Bengal', 7: 'Birman', 8: 'Bombay', 9: 'boxer', 10: 'British_Shorthair',
               11: 'chihuahua', 12: 'Egyptian_Mau', 13: 'english_cocker_spaniel', 14: 'english_setter', 15: 'german_shorthaired', 
               16: 'great_pyrenees', 17: 'havanese', 18: 'japanese_chin', 19: 'keeshond', 20: 'leonberger',
               21: 'Maine_Coon', 22: 'miniature_pinscher', 23: 'newfoundland', 24: 'Persian', 25: 'pomeranian',
               26: 'pug', 27: 'Ragdoll', 28: 'Russian_Blue', 29: 'saint_bernard', 30: 'samoyed',
               31: 'scottish_terrier', 32: 'shiba_inu', 33: 'Siamese', 34: 'Sphynx', 35: 'staffordshire_bull_terrier',
               36: 'wheaten_terrier', 37: 'yorkshire_terrier' } 

  if dataset_is_split:
    # If your dataset is already split, specify each path:
    train_images_dir = '/content/dataset/train/images'
    train_annotations_dir = '/content/dataset/train/annotations'
    val_images_dir = '/content/dataset/validation/images'
    val_annotations_dir = '/content/dataset/validation/annotations'
    test_images_dir = '/content/dataset/test/images'
    test_annotations_dir = '/content/dataset/test/annotations'
  else:
    # If it's NOT split yet, specify the path to all images and annotations
    images_in = '/content/dataset/images'
    annotations_in = '/content/dataset/annotations'
  

In [37]:
!mkdir "/content/dataset/train"

In [38]:
!mkdir "/content/dataset/train/images"

In [39]:
!mkdir "/content/dataset/train/annotations"

In [40]:
!mkdir "/content/dataset/validation"

In [41]:
!mkdir "/content/dataset/validation/images"

In [42]:
!mkdir '/content/dataset/validation/annotations'

In [43]:
def get_class_name_from_filename(file_name):
  """Gets the class name from a file.

  Args:
    file_name: The file name to get the class name from.
               ie. "american_pit_bull_terrier_105.jpg"

  Returns:
    A string of the class name.
  """
  match = re.match(r'([A-Za-z_]+)(_[0-9]+\.jpg)', file_name, re.I)
  return match.groups()[0]

In [44]:
def adjust_to_annotation(data,target_path):
  """Notice that this function normalizes the bounding box coordinates provided
  by the raw data.

  Args:
    data: dict holding PASCAL XML fields for a single image (obtained by
      running dataset_util.recursive_parse_xml_to_dict)
    
    label_map_dict: A map from string label names to integers ids.
    image_subdirectory: String specifying subdirectory within the
      Pascal dataset directory holding the actual image data.
    ignore_difficult_instances: Whether to skip difficult instances in the
      dataset  (default: False).
    faces_only: If True, generates bounding boxes for pet faces.  Otherwise
      generates bounding boxes (as well as segmentations for full pet bodies).
    

  Returns:
    example: The converted tf.Example.

  Raises:
    ValueError: if the image pointed to by data['filename'] is not a valid JPEG
  """
    
  class_name = get_class_name_from_filename(data['filename']) 
  targetXML = open(target_path, 'rt', encoding='UTF8')
  tree = ET.parse(targetXML)

  root = tree.getroot() #
  objcount=0

  for obj in root.findall('object'): 
    if(objcount>1):
      print("objcount>1",targetXML)
    
    name_tag = obj.find('name')
    
    name_tag.text = class_name 
  
  tree.write(target_path)  

  
  

In [ ]:
# xml_file = os.path.join(ANNO_DIR, '2007_000032.xml')

# #XML 파일을 Parsing 하여 Element 생성
# tree = ET.parse(xml_file) #트리를 만든다
# root = tree.getroot() #루트를 가져온다

# #image 관련 정보는 root의 자식으로 존재
# image_name = root.find('filename').text #루트에서 파일 이름 가져온다
# full_image_name = os.path.join(IMAGE_DIR, image_name)
# image_size = root.find('size')
# image_width = int(image_size.find('width').text) #size에서 다시 find를 하면 size의 width를 가져온다
# image_height = int(image_size.find('height').text)

# #파일내에 있는 모든 object Element를 찾음
# objects_list = []
# for obj in root.findall('object'): #모든 object
#     #object element의 자식 element에서 bndbox를 찾음
#     name_tag = obj.find('name')
    
    
    
#     class_name=obj.find('name').text
#     object_dict={'class_name': class_name, 'bndbox_pos':bndbox_pos} #딕셔너리
#     objects_list.append(object_dict) #리스트에 넣는다

In [45]:
import tensorflow.compat.v1 as tf

In [46]:
for idx, example in enumerate(train_examples):
      xml_path = os.path.join(annotations_dir, 'xmls', example + '.xml')
      # print(xml_path)
      if not os.path.exists(xml_path):
        print("no xml...:", xml_path)
        continue

      with tf.gfile.GFile(xml_path, 'r') as fid:
        xml_str = fid.read()
      xml = etree.fromstring(xml_str)
      data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']

      img_path = os.path.join(image_dir, data['filename'])
      # print(img_path)

      trainimgsource = img_path
      trainimgdestination = os.path.join(train_images_dir, data['filename'])
      shutil.copyfile(trainimgsource, trainimgdestination)
      # print(destination)

      trainxmlsource = xml_path
      trainxmldestination = os.path.join(train_annotations_dir, example + '.xml')
      shutil.copyfile(trainxmlsource, trainxmldestination)

      with tf.gfile.GFile(trainxmldestination, 'r') as fid:
        xml_str = fid.read()
      xml = etree.fromstring(xml_str)
      data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
      
      adjust_to_annotation(data, trainxmldestination)
      

/usr/local/lib/python3.8/dist-packages/object_detection/utils/dataset_util.py:83: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


no xml...: /content/dataset/annotations/xmls/Egyptian_Mau_14.xml
no xml...: /content/dataset/annotations/xmls/saint_bernard_15.xml
no xml...: /content/dataset/annotations/xmls/Ragdoll_199.xml
no xml...: /content/dataset/annotations/xmls/Egyptian_Mau_186.xml
no xml...: /content/dataset/annotations/xmls/Bengal_175.xml
no xml...: /content/dataset/annotations/xmls/samoyed_10.xml
no xml...: /content/dataset/annotations/xmls/Egyptian_Mau_156.xml
no xml...: /content/dataset/annotations/xmls/Abyssinian_104.xml


In [47]:
for idx, example in enumerate(val_examples):
      xml_path = os.path.join(annotations_dir, 'xmls', example + '.xml')
      # print(xml_path)
      if not os.path.exists(xml_path):
        print("no xml...:", xml_path)
        continue

      with tf.gfile.GFile(xml_path, 'r') as fid:
        xml_str = fid.read()
      xml = etree.fromstring(xml_str)
      data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']

      img_path = os.path.join(image_dir, data['filename'])
      # print(img_path)

      valimgsource = img_path
      valimgdestination = os.path.join(val_images_dir, data['filename'])
      shutil.copyfile(valimgsource, valimgdestination)
      # print(destination)

      valxmlsource = xml_path
      valxmldestination = os.path.join(val_annotations_dir, example + '.xml')
      shutil.copyfile(valxmlsource, valxmldestination)

      with tf.gfile.GFile(valxmldestination, 'r') as fid:
        xml_str = fid.read()
      xml = etree.fromstring(xml_str)
      data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
      adjust_to_annotation(data, valxmldestination)
      

no xml...: /content/dataset/annotations/xmls/Bengal_111.xml


#내부적으로 tfrecord만드는 코드

In [48]:
# We need to instantiate a separate DataLoader for each split dataset
if use_custom_dataset:
  if dataset_is_split:
    train_data = object_detector.DataLoader.from_pascal_voc(
        train_images_dir, train_annotations_dir, label_map=label_map)
    validation_data = object_detector.DataLoader.from_pascal_voc(
        val_images_dir, val_annotations_dir, label_map=label_map)
    # test_data = object_detector.DataLoader.from_pascal_voc(
    #     test_images_dir, test_annotations_dir, label_map=label_map)
  else:
    train_dir, val_dir, test_dir = split_dataset(images_in, annotations_in,
                                                 val_split=0.2, test_split=0.2,
                                                 out_path='split-dataset')
    train_data = object_detector.DataLoader.from_pascal_voc(
        os.path.join(train_dir, 'images'),
        os.path.join(train_dir, 'annotations'), label_map=label_map)
    validation_data = object_detector.DataLoader.from_pascal_voc(
        os.path.join(val_dir, 'images'),
        os.path.join(val_dir, 'annotations'), label_map=label_map)
    test_data = object_detector.DataLoader.from_pascal_voc(
        os.path.join(test_dir, 'images'),
        os.path.join(test_dir, 'annotations'), label_map=label_map)
    
  print(f'train count: {len(train_data)}')
  print(f'validation count: {len(validation_data)}')
  # print(f'test count: {len(test_data)}')

train count: 2568
validation count: 1103


In [49]:
import tensorflow as tf

In [50]:
spec = object_detector.EfficientDetLite1Spec()

In [52]:
model = object_detector.create(train_data=train_data, 
                               model_spec=spec, 
                               validation_data=validation_data, 
                               epochs=250, 
                               batch_size=32, 
                               train_whole_model=True)

Epoch 1/250
80/80 [==============================] - 142s 1s/step - det_loss: 1.9109 - cls_loss: 1.5275 - box_loss: 0.0077 - reg_l2_loss: 0.0768 - loss: 1.9878 - learning_rate: 0.0240 - gradient_norm: 1.0333 - val_det_loss: 1.6057 - val_cls_loss: 1.3966 - val_box_loss: 0.0042 - val_reg_l2_loss: 0.0768 - val_loss: 1.6826
Epoch 2/250
80/80 [==============================] - 101s 1s/step - det_loss: 1.2036 - cls_loss: 1.0376 - box_loss: 0.0033 - reg_l2_loss: 0.0771 - loss: 1.2807 - learning_rate: 0.0400 - gradient_norm: 1.6616 - val_det_loss: 1.1398 - val_cls_loss: 0.9130 - val_box_loss: 0.0045 - val_reg_l2_loss: 0.0774 - val_loss: 1.2172
Epoch 3/250
80/80 [==============================] - 102s 1s/step - det_loss: 0.8556 - cls_loss: 0.7318 - box_loss: 0.0025 - reg_l2_loss: 0.0776 - loss: 0.9332 - learning_rate: 0.0400 - gradient_norm: 1.6184 - val_det_loss: 1.0621 - val_cls_loss: 0.8450 - val_box_loss: 0.0043 - val_reg_l2_loss: 0.0778 - val_loss: 1.1399
Epoch 4/250
80/80 [===============

In [53]:
model.evaluate(validation_data)

18/18 [==============================] - 92s 2s/step



{'AP': 0.7659774,
 'AP50': 0.8734385,
 'AP75': 0.8492359,
 'APs': -1.0,
 'APm': 0.73991716,
 'APl': 0.7692563,
 'ARmax1': 0.8199018,
 'ARmax10': 0.84293085,
 'ARmax100': 0.84320825,
 'ARs': -1.0,
 'ARm': 0.8067348,
 'ARl': 0.8465883,
 'AP_/Abyssinian': 0.7992612,
 'AP_/american_bulldog': 0.7103237,
 'AP_/american_pit_bull_terrier': 0.5672764,
 'AP_/basset_hound': 0.77281874,
 'AP_/beagle': 0.6176051,
 'AP_/Bengal': 0.6782444,
 'AP_/Birman': 0.7811429,
 'AP_/Bombay': 0.8873229,
 'AP_/boxer': 0.6837058,
 'AP_/British_Shorthair': 0.89174813,
 'AP_/chihuahua': 0.6850277,
 'AP_/Egyptian_Mau': 0.69568163,
 'AP_/english_cocker_spaniel': 0.69962513,
 'AP_/english_setter': 0.7353636,
 'AP_/german_shorthaired': 0.7770208,
 'AP_/great_pyrenees': 0.7575933,
 'AP_/havanese': 0.74346954,
 'AP_/japanese_chin': 0.7394185,
 'AP_/keeshond': 0.80908644,
 'AP_/leonberger': 0.83687794,
 'AP_/Maine_Coon': 0.8337249,
 'AP_/miniature_pinscher': 0.60170984,
 'AP_/newfoundland': 0.7356333,
 'AP_/Persian': 0.805

In [17]:
TFLITE_FILENAME = 'efficientdet-lite-oxfordIII.tflite'
LABELS_FILENAME = 'oxfordIII-labels.txt'

In [18]:
model.export(export_dir='.', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

NameError: ignored

In [56]:
model.evaluate_tflite(TFLITE_FILENAME, validation_data)

1103/1103 [==============================] - 5771s 5s/step



{'AP': 0.74392307,
 'AP50': 0.8571324,
 'AP75': 0.8319684,
 'APs': -1.0,
 'APm': 0.7127653,
 'APl': 0.74711996,
 'ARmax1': 0.7922268,
 'ARmax10': 0.7922268,
 'ARmax100': 0.7922268,
 'ARs': -1.0,
 'ARm': 0.74156445,
 'ARl': 0.7945301,
 'AP_/Abyssinian': 0.8001138,
 'AP_/american_bulldog': 0.6943849,
 'AP_/american_pit_bull_terrier': 0.48444486,
 'AP_/basset_hound': 0.756464,
 'AP_/beagle': 0.57883036,
 'AP_/Bengal': 0.6791395,
 'AP_/Birman': 0.7611382,
 'AP_/Bombay': 0.8831512,
 'AP_/boxer': 0.6478774,
 'AP_/British_Shorthair': 0.8671419,
 'AP_/chihuahua': 0.6801545,
 'AP_/Egyptian_Mau': 0.6272264,
 'AP_/english_cocker_spaniel': 0.661163,
 'AP_/english_setter': 0.68414414,
 'AP_/german_shorthaired': 0.75384426,
 'AP_/great_pyrenees': 0.76901245,
 'AP_/havanese': 0.7174994,
 'AP_/japanese_chin': 0.7111762,
 'AP_/keeshond': 0.79701996,
 'AP_/leonberger': 0.8142334,
 'AP_/Maine_Coon': 0.814164,
 'AP_/miniature_pinscher': 0.5675285,
 'AP_/newfoundland': 0.70732355,
 'AP_/Persian': 0.8065308

In [ ]:
import random

# If you're using a custom dataset, we take a random image from the test set:
if use_custom_dataset:
  images_path = test_images_dir if dataset_is_split else os.path.join(test_dir, "images")
  filenames = os.listdir(os.path.join(images_path))
  random_index = random.randint(0,len(filenames)-1)
  INPUT_IMAGE = os.path.join(images_path, filenames[random_index])
# else:
#   # Download a test salad image
#   INPUT_IMAGE = 'salad-test.jpg'
#   DOWNLOAD_URL = "https://storage.googleapis.com/cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg"
#   !wget -q -O $INPUT_IMAGE $DOWNLOAD_URL

In [ ]:
INPUT_IMAGE="/content/dataset/validation/images/Abyssinian_183.jpg"

In [ ]:
! python3 -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral

In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

import tflite_runtime.interpreter as tflite 
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

def draw_objects(draw, objs, scale_factor, labels):
  """Draws the bounding box and label for each object."""
  COLORS = np.random.randint(0, 255, size=(len(labels), 3), dtype=np.uint8)
  for obj in objs:
    bbox = obj.bbox
    color = tuple(int(c) for c in COLORS[obj.id])
    draw.rectangle([(bbox.xmin * scale_factor, bbox.ymin * scale_factor),
                    (bbox.xmax * scale_factor, bbox.ymax * scale_factor)],
                   outline=color, width=3)
    font = ImageFont.truetype("LiberationSans-Regular.ttf", size=15)
    draw.text((bbox.xmin * scale_factor + 4, bbox.ymin * scale_factor + 4),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill=color, font=font)

# Load the TF Lite model
labels = read_label_file(LABELS_FILENAME)
interpreter = tflite.Interpreter(TFLITE_FILENAME)
interpreter.allocate_tensors()

# Resize the image for input
image = Image.open(INPUT_IMAGE)
_, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

# Run inference
interpreter.invoke()
objs = detect.get_objects(interpreter, score_threshold=0.4, image_scale=scale)

# Resize again to a reasonable size for display
display_width = 500
scale_factor = display_width / image.width
height_ratio = image.height / image.width
image = image.resize((display_width, int(display_width * height_ratio)))
draw_objects(ImageDraw.Draw(image), objs, scale_factor, labels)
image